# Abstract Base Class

- not all classes are concrete with attributes and methods
- some classes are missing details and are called abstract classes
- abstract classes aren't directly usable themselves; but are meant to be inherited to create concrete classes
- Note: *base* class and *super* class are used as synonyms
- abstract class helps us create abstraction and make sure that child/concrete classes have replaced the abstraction

![ABC Figure](resources/ABC.png)

- learn:
    - creating an abstract base class
    - ABCs and type hints
    - the *collections.abc* module
    - operator overloading
    - extending built-ins
    - Metaclasses
    
## Creating an abstract base class (ABC)

- define a media player as an *abstraction*
- each unique kind of media file format can provide a *concrete* implementation of the abstraction
- use `pass` or `...` keywords to complete the function definition
- doc string also syntactically completes the function definition

In [ ]:
import abc

class MediaLoader(abc.ABC):
    @abc.abstractmethod
    def play(self) -> None:
        ... # ellipsis
        
    @property
    @abc.abstractmethod
    def ext(self) -> str:
        pass # placeholder

In [ ]:
# special attribute of class shows you a set of all the abstract methods
MediaLoader.__abstractmethods__

In [ ]:
# let's see what happens if we implement a subclass
# without providing details for abstract methods
class MP3(MediaLoader):
    pass

In [ ]:
mp3 = MP3()

In [ ]:
class Ogg(MediaLoader):
    ext = '.ogg' # property
    
    def play(self): # method
        pass

In [ ]:
ogg = Ogg()

In [ ]:
ogg.ext

## The ABCs of collections

- collections module contains *abc* to extend and create custom containers
- *Collection* is an extension of an even more fundamental abstraction, *Container*
- the `collection.abc` module provides abstract base class definitions for Python built-in collections such as: list, dict, set, etc.
- we can use the definitions to build our own unique data structures
- e.g., `dict` concrete container has the following class hierarchy
![ABC Containers](resources/abc_containers.png)


In [ ]:
import collections.abc

In [ ]:
help(collections.abc)

In [ ]:
# let's see the APIs of dict to replicate them in our own example below
help(dict)

In [ ]:
from collections.abc import Container

In [ ]:
Container.__abstractmethods__

In [ ]:
help(Container.__contains__)

In [ ]:
class OddIntegers:
    def __contains__(self, x: int) -> bool:
        return x%2 != 0

In [ ]:
odd = OddIntegers()

In [ ]:
# Though OddIntegers doesn't inherit from Container, 
# it looks like Container because of __contains__ -- duck typing!
isinstance(odd, Container)

In [ ]:
issubclass(OddIntegers, Container)

In [ ]:
# any class that has __contains__ method is a container
# in operator is overloaded which calls __contains___
2 in odd

In [ ]:
3 in odd

### Implement an Immutable Mapping container

- **Protocol** - is how the duck typing works:
    - when two classes have the same set of methods, they both adhere to a common protocol    
- let's extend the `collections.abc` to define our own dictionary-like mapping (look-up) object
- we'll use the following type hint for *mypy*

```python
BaseMapping = abc.Mapping[Comparable, Any]
```

- key type is Comparable, so we can compare and order the keys
    - searching a list in order is much faster than an unordered list
- value type is Any object
- we'll use same initializers as built-in dict is built from a mapping or a sequence of pairs as shown below
- use `bisect_left` or `bisect_right` binary search functions in `bisect` module to keep the sorted list of keys
- **bisect_left(alist, x)**
    - returns the leftmost index where x should be inserted to keep `alist` sorted
    - ff `x` is already present, the insertion point will be before (to the left of) any existing entries of x
- **bisect_right(alist, x)**
    - returns the rightmost index where x should be inserted to keep `alist` sorted
    - if `x` is already present, the insertion point will be after the existing entries of x
- what is the worst cast time complexity (big O notation) of binary search?

In [ ]:
import bisect

In [ ]:
alist = [1, 3, 3, 3, 5]

In [ ]:
i = bisect.bisect_left(alist, 3)
print(i)

In [ ]:
print(bisect.bisect_right(alist, 3))

In [ ]:
# dict can be generated by initilizing with various data types
x = dict({"a": 42, "b": 7, "c": 6})

In [ ]:
y = dict((("a", 42), ("b", 7), ("c", 6)))

In [ ]:
z = dict([("a", 42), ("b", 7), ("c", 6)])

In [ ]:
x == y == z

In [ ]:
# let's define a Comparable class that'll be used as a type in the Lookup class definition

from typing import Protocol, Any

class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool: ...
    def __ne__(self, other: Any) -> bool: ...
    def __le__(self, other: Any) -> bool: ...
    def __lt__(self, other: Any) -> bool: ...
    def __ge__(self, other: Any) -> bool: ...
    def __gt__(self, other: Any) -> bool: ...

In [ ]:
from __future__ import annotations
from collections import abc
from typing import Protocol, Any, overload, Union
import bisect
from typing import Iterator, Iterable, Sequence, Mapping

BaseMapping = abc.Mapping[Comparable, Any]

class Lookup(BaseMapping):
    
    # To make it clear to mypy, we need to provide overloaded method definitions using @overload
    @overload
    def __init__(self, source: Iterable[tuple[Comparable, Any]]) -> None:
        ...
        
    @overload
    def __init__(self, source: BaseMapping) -> None:
        ...
        
    def __init__(self, 
                source: Union[Iterable[tuple[Comparable, Any]], 
                              BaseMapping, None] = None,
    ) -> None:
        sorted_pairs: Sequence[tuple[Comparable, Any]]
        if isinstance(source, Sequence):
            sorted_pairs = sorted(source)
        elif isinstance(source, abc.Mapping):
            sorted_pairs = sorted(source.items())
        else:
            sorted_pairs = []
        self.key_list = [p[0] for p in sorted_pairs]
        self.value_list = [p[1] for p in sorted_pairs]
        assert len(self.key_list) == len(self.value_list)
        
    # Abstract methods from base classes
    def __len__(self) -> int:
        return len(self.key_list)
    
    def __iter__(self) -> Iterator[Comparable]:
        return iter(self.key_list)
    
    def __contains__(self, key: object) -> bool:
        # can use bisect_right or left
        index = bisect.bisect_left(self.key_list, key)
        return key == self.key_list[index]
    
    def __getitem__(self, key:Comparable) -> Any:
        index = bisect.bisect_left(self.key_list, key)
        if key == self.key_list[index]:
            return self.value_list[index]
        raise KeyError(key)
        

In [ ]:
look = Lookup({'a': 1, 'b': 2, 'c': 3, 'z': 26})

In [ ]:
'z' in look

In [ ]:
'f' in look

In [ ]:
look['z']

In [ ]:
look['m']

In [ ]:
for k in look:
    print(k, '->', look[k])

In [ ]:
list(look.items())

In [ ]:
# Since keys are sorted, they must be comparable
x = Lookup([
    ('a', 'Apple'),
    ('b', 'Ball'),
    ('uno', 'one'),
    ('1', 'One')
])

In [ ]:
x['a']

In [ ]:
x['10']

In [ ]:
# Immutable dictionary! __setitem__ is not implemented!
x['10'] = 'Ten'

### Rules to extend abc

- Find a class that does most of what you need
- Identify the abstract methods in collections.abc definitions
    - look at the help docs, source code, etc.
- Subclass the abstract class, filling in the missing methods
- use **mypy** and **unittest** to make sure abstract methods are implemented and working correctly

## Creating your own abc 

- simulating a game that involves rolling of polyhedral dice
    - dices with four, eight, twelve and twenty sides
    
  

In [2]:
import abc

class Die(abc.ABC):
    def __init__(self) -> None:
        self.face: int
        self.roll()
        
    @abc.abstractmethod
    def roll(self) -> None:
        ...
        
    def __repr__(self) -> str:
        return f'{self.face}'
    

In [3]:
Die.__abstractmethods__

frozenset({'roll'})

In [4]:
# can also check if a method is abstract method
Die.roll.__isabstractmethod__

True

In [5]:
import random

class D4(Die):
    def roll(self) -> None:
        self.face = random.choice((1, 2, 3, 4))

class D6(Die):
    def roll(self) -> None:
        self.face = random.randint(1, 6)
        

In [6]:
class Dice(abc.ABC):
    def __init__(self, n: int, die_class: Die) -> None:
        self.dice = [die_class() for _ in range(n)]
    
    @abc.abstractmethod
    def roll(self) -> None:
        ...
        
    @property
    def total(self) ->int:
        return sum(d.face for d in self.dice)

In [7]:
# The subclass implements the roll-all-the-dice rule
class SimpleDice(Dice):
    def roll(self) -> None:
        for d in self.dice:
            d.roll()

In [8]:
sd = SimpleDice(6, D6)

In [9]:
# roll the dice a few times to see the random total
sd.roll()

In [10]:
sd.total

14

### Yacht dice game

- 5 6-sided dice are used
- players take turns (usually 12 turns total)
- on your turn:
    - roll all 5 dice
    - you can re-roll any number of dice up to 2 more times (max 3 rolls per turn)
    - choose a scoring category to place your final dice in
    - once a category is used, it can’t be used again

In [ ]:
from typing import Iterable

class YachtDice(Dice):
    def __init__(self) -> None:
        super().__init__(5, D6)
        # keep track of saved dice positions; these dice are not rolled
        self.saved: Set[int] = set()
            
    # save dice positions in the saved set
    # save dice positions are not rolled
    def saving(self, positions: Iterable[int]) -> "YactDice":
        if not all(0 <= n < 5 for n in positions):
            raise ValueError("Invalid position")
        self.saved = set(positions)
        return self
    
    def roll(self) -> None:
        for n, d in enumerate(self.dice):
            if n not in self.saved:
                d.roll()
                
        self.saved = set()

In [12]:
YachtDice.__abstractmethods__

frozenset()

In [13]:
yd = YachtDice()

In [14]:
yd.roll()

In [15]:
yd.dice

[5, 2, 5, 4, 2]

In [16]:
yd.saving([0, 3]).roll()

In [17]:
yd.dice

[5, 5, 1, 4, 2]

In [18]:
yd.saving([1, 2, 4]).roll()

In [19]:
yd.dice

[5, 5, 1, 3, 2]

In [20]:
yd.total

16

## Exercises

- solve the following Kattis problems using ABC
- must use the following Kattis ABC class
- see demo: [https://github.com/rambasnet/Object-Oriented-Programming-Design-Patterns/tree/main/demo-assignments/A2-ABC/egypt](https://github.com/rambasnet/Object-Oriented-Programming-Design-Patterns/tree/main/demo-assignments/A2-ABC/egypt)

```python

from abc import ABC, abstractmethod
from typing import Any

class Kattis(ABC):
	"""
	Solution ABC class for Kattis problems
	"""
	def __init__(self, data_source: Any) -> None:
		"""
		Constructor
        :param data_source: input data source object
        :return: None
		"""
		self._input_source: Any = data_source
		self._data: Any = None
		self._answer: Any = None

	@abstractmethod
	def read_input(self) -> None:
		"""
		Reads the data from the given source
		:return: None
		"""
		...

	@property
	@abstractmethod
	def data(self) -> Any:
		"""
		Returns the data
		:return: data
		"""
		...

	@property
	@abstractmethod
	def answer(self) -> Any:
		"""
		Returns the answer
		:return: answer
		"""
		...

	@abstractmethod
	def solve(self) -> None:
		"""
		Solves the problem
		:return: None
		"""
		...

	@abstractmethod
	def print_answer(self) -> None:
		"""
		Prints the answer
		:return: None
		"""
		...
		
```

1. Create a new data structure called Teque -- the definition of which can be found here: https://open.kattis.com/problems/teque
    - Solve the problem using the new Teque type defined by extending Deque or similar abc
    - Teque must implent push_back, push_front, push_middle and get interfaces at a minimum
    - Solution will be accepted if at least all but last 2 cases are accepted. If you receive TLE in the last or 2nd last test case on Kattis, you will receive full credit.
    
```python

from typing import Deque
from collections.abc import Sequence

class Teque(Sequence[int]):
    def __init__(self) -> None:
        self._q1: Deque = deque()
        self._q2: Deque = deque()

    def __len__(self) -> int:
        pass

    @overload
    def __getitem__(self, i: int) -> int: ...

    @overload
    def __getitem__(self, i: slice) -> Teque: ...

    def __getitem__(self, i: Union[int, slice]) -> Union[int, Teque]:
        if isinstance(i, slice):
            return Teque()
        if i < len(self._q1):
            return self._q1[i]
        else:
            return self._q2[i-len(self._q1)]

    def __iter__(self) -> Iterator[int]:
        for x in self._q1:
            yield x
        for x in self._q2:
            yield x

    def __reversed__(self) -> Iterator[int]:
        pass

    def insert(self, i, x) -> None:
        pass

    def push_back(self, x) -> None:
        pass

    def push_front(self, x) -> None:
        pass

    def push_middle(self, x) -> None:
        pass

    def get(self, i) -> int:
        return self[i]
```
    
2. Solve Kattis CD problem - https://open.kattis.com/problems/cd using ABC and OOD
    - Create a new data structure called CD inherited from MutableSequence.
    - CD must have `common()` interface method to find the intersection between other CD object
    - `__and__()` method overloads `&` intersection operator
    - e.g., jack & jill
    - Must use the following class definition

```python
from __future__ import annotations
from collections.abc import MutableSequence
from typing import List, Iterator, Union, overload

class CD(MutableSequence[int]):
	def __init__(self, count: int = 0, maxCount: int = 1_000_000) -> None:
		self._count: int = count
		self._ids : List[int] = [0]*maxCount

	def __len__(self) -> int:
		return len(self._ids)

	@overload
	def __getitem__(self, idx: int) -> int: ...

	@overload
	def __getitem__(self, idx: slice) -> CD: ...

	def __getitem__(self, idx: Union[int, slice]) -> Union[int, CD]:
		if isinstance(idx, slice):
			return CD() 
		return self._ids[idx]

	@overload
	def __setitem__(self, idx:int, x: int) -> None: ...

	@overload
	def __setitem__(self, idx: slice, x: Iterable[int]) -> None: ...

	def __setitem__(self, idx: Union[int, slice], x: Union[int, Iterable]) -> None:
		if isinstance(idx, int) and isinstance(x, int):
			self._ids[idx] = x
		elif isinstance(idx, slice) or isinstance(x, Iterable):
			raise NotImplementedError

	def __delitem__(self, i) -> None:
		del self._ids[i]

	def __iter__(self) -> Iterator[int]:
		for x in self._ids:
			yield x

	def __reversed__(self) -> Iterator[int]:
		for x in reversed(self._ids):
            yield x

	def __str__(self) -> str:
		return str(self._ids)
    
    def insert(self, i: int, x: int) -> None:
		self._ids[i] = x

	@property
	def last(self) -> int:
		pass

	@property
	def count(self) -> int:
		return self._count

	@count.setter
	def count(self, count: int) -> None:
		self._count = count
    
	def __and__(self, other: 'CD') -> int:
		common = 0
		i = 0
		j = 0
		# FIXME: find the common ids between this and other
        - loop from 0 to count
            - if this id at i is larger than the last id of other or vice versa exit loop
            - if this id at i equals other id at j, increment common, i and j
            - else if this id at i less than other id at j, increment i
            - else increment j
		return common
```